# Auto-encoding of Julia Source Code

We investigate the ability of deep neural network architectures to capture key syntactic and semantic information in Julia source code using relatively low-dimensional representations. These representations can be potentially very powerful tools for automated program analysis and curation, especially across heterogeneous research fields and repositories. 

There is in fact a wealth of [research](https://github.com/src-d/awesome-machine-learning-on-source-code) in this area, though to date there has been no application of these ideas or approaches to the Julia language itself. In this experiment, we study the makeup and patterns of the base Julia language source code in the hopes of identifying futher avenues of research. 

We use the Keras deep learning API to construct our autoencoding model, and Numpy and Pandas libraries to manage the data inputs and outputs. 

In [ ]:
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.layers import Input, GRU, RepeatVector, Activation, CuDNNGRU
from keras.layers import Dense, BatchNormalization, Embedding
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import numpy as np
import pandas as pd


The shape of the lower-dimensional representation we wish to create is a somewhat arbitrary choice; we chose 64 dimensions as a reasonably rich but still manageable number. Identifying optimal dimensionality is an area for further research. 

We read our saved code snippets and source filepath labels in to a Pandas DataFrame. We also truncate any path information above the Julia repo itself as irrelevant (and necessarily uniformly repeated). Given large number of unique sources filepaths (546) and the low number of snippets in the median source file (19), we compute two aggregated groups of source files: top-level folders in the Julia repo and second-level divisions, which include both folders (with multiple files and folders within them) as well as single source code files in the top level folders themselves. 

There are 7 top level folders with a wide range of amount of code within each, and 273 second-level divisions:

```
test               9391
stdlib             7846
base               7486
doc                  30
contrib              25
src                   7
etc                   2

base_gcutils          5
base_array          170
test_meta            70
test_spawn           85
test_project          8
test_stacktraces     12
base_sysinfo         34
[...]```

In [ ]:
latent_dim = 64

with open("all_funcs.csv", "r") as f: 
    funcs = f.read()

funcs = funcs.split(".jl\n")
funcs = funcs[:-1] # remove trailing empty item
funcs = pd.DataFrame([x.rsplit("\t",1) for x in funcs])
funcs.columns = ['code','source']
funcs = funcs[funcs.code.apply(lambda x: len(x)<=500)]
funcs.reset_index(drop=True, inplace=True)

funcs.source = funcs.source.apply(lambda x: x[x.index("julia/")+6:])
funcs["top_folder"] = funcs.source.apply(lambda x: x[:x.index("/")])
funcs['top2'] = funcs.source.apply(lambda x: '_'.join(x.split("/")[:2]))


We begin with a common sequence-to-sequence modeling approach as represented by [recent work](https://towardsdatascience.com/how-to-create-data-products-that-are-magical-using-sequence-to-sequence-models-703f86a231f8) at GitHub itself. This system was originally built to summarize Github issues, using the issue text as input sequences and issue titles as the target sequences. We adapt this to use as an auto-encoding system by using the same source code snippets as both inputs and targets. 
<img src="../img/autoenc_diagram.png" />

_(Please note, if you have Keras v. 2.2.4 installed that there is an issue with the accuracy calculation (https://github.com/keras-team/keras/issues/11749). This is resolvable by upgrading to the latest version available via git or by downgrading to an earlier version of Keras. Given that the problem is already fixed in the latest unreleased code, we expect this issue to be resolved in the next pip-able release as well (i.e., 2.2.5).)_

We define two utility functions to aid in our encoding: `chars_to_indices()` which translates Julia source code into integers representing each character, and `ae_models` which build our autoencoder architecture. This second function returns two models - the full autoencoder, as well as the encoder sub-component. We use this second model to encode our Julia source code sequences after training is complete.

In [1]:
def chars_to_indices(data, tok=None, max_len=None):
    if max_len is None:
        max_len = max(data.apply(lambda x: len(x)))

    if tok is None:
        tok = Tokenizer(num_words=None, 
                        filters="", 
                        lower=False, 
                        split='', 
                        char_level=True)

    data = data.values
    tok.fit_on_texts(data)
    sequences = tok.texts_to_sequences(data)
    sequences = pad_sequences(sequences, 
                              maxlen=max_len, 
                              padding='post')
    sequences = np.array(sequences, dtype='int16')

    return sequences, tok

def ae_models(maxlen, latent_dim, N, use_gpu=False):
    inputs = Input((maxlen,), name='Encoder_Inputs')
    encoded = Embedding(N, 
                        latent_dim, 
                        name='Char_Embedding', 
                        mask_zero=False)(inputs)
    encoded = BatchNormalization(name='BatchNorm_Encoder')(encoded)

    if use_gpu:
        _, state_h = CuDNNGRU(latent_dim, return_state=True)(encoded)
    else:
        _, state_h = GRU(latent_dim, return_state=True)(encoded)

    enc = Model(inputs=inputs, outputs=state_h, name='Encoder_Model')
    enc_out = enc(inputs)

    dec_inputs = Input(shape=(None,), name='Decoder_Inputs')
    decoded = Embedding(N, 
                        latent_dim, 
                        name='Decoder_Embedding', 
                        mask_zero=False)(dec_inputs)
    decoded = BatchNormalization(name='BatchNorm_Decoder_1')(decoded)

    if use_gpu:
        dec_out, _ = CuDNNGRU(latent_dim, 
                              return_state=True, 
                              return_sequences=True)(decoded, initial_state=enc_out)
    else:
        dec_out, _ = GRU(latent_dim, 
                         return_state=True, 
                         return_sequences=True)(decoded, initial_state=enc_out)

    dec_out = BatchNormalization(name='BatchNorm_Decoder_2')(dec_out)
    dec_out = Dense(N, activation='softmax', name='Final_Out')(dec_out)

    sequence_autoencoder = Model(inputs=[inputs, dec_inputs], outputs=dec_out)

    return sequence_autoencoder, enc


We convert our Julia code snippets to vectors of integers, tokenized at the character level. E.g.,

```Julia
a::BitSet ⊊ b::BitSet = begin
        #= none:414 =#
        a <= b && a != b
    end
```
    
becomes:

```Python
array([ 11,   8,   8,  52,   9,   4,  48,   2,   4,   1, 135,   1,  28,
         8,   8,  52,   9,   4,  48,   2,   4,   1,   5,   1,  28,   2,
        24,   9,   3,  13,   1,   1,   1,   1,   1,   1,   1,   1,  10,
         5,   1,   3,   6,   3,   2,   8,  37,  21,  37,   1,   5,  10,
        13,   1,   1,   1,   1,   1,   1,   1,   1,  11,   1,  65,   5,
         1,  28,   1,  76,  76,   1,  11,   1,  67,   5,   1,  28,  13,
         1,   1,   1,   1,   2,   3,  17])```

There are 235 unique characters in our Julia source code data (343 in the entire source code corpus, though over 100 of these are contained solely in a handful of excessively long expressions that were excluded for this experiment). To leverage teacher forcing in our seq2seq autoencoder training, we include the original input sequence as an input to the decoder portion of the autoencoder, and our target output is the the same input sequence one character offset. 


In [ ]:
seqs, tok = chars_to_indices(funcs.iloc[:,0])
N = len(np.unique(seqs))

decoder_inputs = seqs[:,  :-1]
Y = seqs[:, 1:  ]


We then build our model. The system can be set to run on a GPU or a CPU, depending on the hardware available. On an Nvidia Quadro P4000 training this model takes roughly 50 seconds per epoch.

In [6]:
# autoencoder, enc = ae_models(max_len, 64, N, use_gpu=True)
autoencoder, enc = ae_models(500, 64, 235, use_gpu=False)
autoencoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder_Inputs (InputLayer)     (None, None)         0                                            
__________________________________________________________________________________________________
Decoder_Embedding (Embedding)   (None, None, 64)     15040       Decoder_Inputs[0][0]             
__________________________________________________________________________________________________
Encoder_Inputs (InputLayer)     (None, 500)          0                                            
__________________________________________________________________________________________________
BatchNorm_Decoder_1 (BatchNorma (None, None, 64)     256         Decoder_Embedding[0][0]          
__________________________________________________________________________________________________
Encoder_Mo

We found that Adam optimization with amsgrad gave the best results for training efficiency and accuracy. We use sparse categorical crossentropy as our loss function, with the aim to reproduce input code snippets as exactly as possible in our outputs, character by character, over the 235-character "vocabulary."

We train for up to 100 epochs, using a batch size of 32 and validating on 12% of the total data. We also use early stopping to settle on an optimal set of model weights should learning level out before the 100th epoch. In various tests this was always the case, with our final model achieving a 93% level of per-character accuracy on the 44th epoch. In some cases it is useful to reduce the learning rate to 0.0001 after the first training session has stopped, but in our experiment this did not improve the model's performance.

In [ ]:
opt = Adam(lr=0.001, amsgrad=True)

autoencoder.compile(loss='sparse_categorical_crossentropy',
                    optimizer=opt,
                    metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_acc',
                          min_delta=0.0001,
                          patience=10,
                          verbose=1,
                          mode='auto',
                          restore_best_weights=True)

autoencoder.fit([seqs, decoder_inputs],
                np.expand_dims(Y, -1),
                epochs = 100,
                batch_size = 32,
                validation_split=0.12,
                callbacks=[early_stop],
                shuffle=True)

autoencoder.save("autoencoder.h5")
enc.save("encoder.h5")


We encode our Julia source code snippet sequences with our newly trained encoder model. This gives us a two-dimensional Numpy array of shape (24787, 64) - each snippet being one row, represented as a vector of 64 float values.

In [ ]:
encoded_reps = enc.predict(seqs)
encoded_reps = pd.DataFrame(encoded_reps)

encoded_reps.to_csv("encoded_reps.csv", index=False)


## Visualization and Analysis
We use Python's `matplotlib` library for plotting our visualizations, and [`UMAP`](https://arxiv.org/abs/1802.03426) (Uniform Manifold Approximation and Projection for Dimension Reduction) to represent our findings in a two-dimensional space. We also include code to produce three-dimensional interactive visualizations which can be even more informative, but the interactivity is beyond the scope of this static notebook documentation.

UMAP is a novel manifold learning technique for dimension reduction constructed from a theoretical framework based in Riemannian geometry and algebraic topology. The UMAP algorithm is competitive with the popular t-SNE method for visualization quality, while seeking to preserve more of the global structure with superior run time performance.

For visualizing our findings here, we limit our test to the "base" top-level folder (7486 code snippets), and label our cases by their second-level folder/file. We further limit our test cases to those labels with at least 100 examples for better visualization and comparison (21 categories).


In [ ]:
from matplotlib import colors as mcolors
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
import sklearn.cluster as cluster
from umap import UMAP

X_test = encoded_reps[funcs.top_folder=="base"]
Y_test = funcs.top2[funcs.top_folder=="base"]
code_test = funcs.code[funcs.top_folder=="base"]
top_cats = list(Y_test.value_counts()[Y_test.value_counts()>=100].index)

X_test = X_test[Y_test.apply(lambda x: x in top_cats)]
code_test = code_test[Y_test.apply(lambda x: x in top_cats)]
Y_test = Y_test[Y_test.apply(lambda x: x in top_cats)]


We use cosine distance to compare the 64-dimensional vectors for our code snippets, and compute UMAP projections in two- and three-dimensional representations. Cosine distance is a measure of distnace between two vectors of an inner product space that measures the cosine of the angle between them, and is one of the most common distance metrics for comparing sequences in machine learning. 

In [ ]:
reducer = UMAP(random_state=42, 
               metric='cosine', 
               n_neighbors=30, 
               n_components=2)
embedding = reducer.fit_transform(X_test)

reducer_3d = UMAP(random_state=42, 
                  metric='cosine', 
                  n_neighbors=30, 
                  n_components=3)
embedding_3d = reducer_3d.fit_transform(X_test)


We compute silhouette scores, which measure cohesion vs. overlap in multi-dimensional clusters on a scale from -1 (no separation and perfect overlap) to 1 (perfectly separated and compact clusters), both for our latent 64-dimensional space and the two-dimensional UMAP representation for our 21 test categories. We then visualize the UMAP representation (optionally in three dimensions).

In [ ]:
sils = silhouette_samples(X_test, Y_test, metric='cosine')
clusts = pd.concat([X_test.reset_index(drop=True), 
                    Y_test.reset_index(drop=True), 
                    pd.Series(sils)], axis=1, ignore_index=True)
centroids = clusts.groupby(64).agg('mean').sort_values(65)

sils2 = silhouette_samples(embedding, Y_test, metric='cosine')
clusts2 = pd.concat([X_test.reset_index(drop=True), 
                     Y_test.reset_index(drop=True), 
                     pd.Series(sils2)], axis=1, ignore_index=True)
centroids2 = clusts2.groupby(64).agg('mean').sort_values(65)

src = list(centroids.index)

colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
by_hsv = sorted((tuple(mcolors.rgb_to_hsv(mcolors.to_rgba(color)[:3])), name)
                for name, color in colors.items())
sorted_names = [name for hsv, name in by_hsv]

NUM_COLORS = len(src)
step = int(np.floor(len(sorted_names)/NUM_COLORS))
my_cols = [sorted_names[i] 
           for i in range(0, len(sorted_names), step)]

fig, ax = plt.subplots(figsize=(12, 10))
for i, s in enumerate(src):
    ax.scatter(embedding[Y_test==s, 0], 
                embedding[Y_test==s, 1], 
                c=my_cols[i], 
                linewidths=0.1,
                edgecolors='k',
                label=s)
plt.setp(ax, xticks=[], yticks=[]) 
plt.title("Julia source code data embedded into two dimensions by UMAP", 
          fontsize=18) 
plt.legend(loc="upper left", bbox_to_anchor=(1,1))
plt.subplots_adjust(right=0.75)
plt.show()

# #
# # 3d Plot
# # 
# fig, ax = plt.subplots(figsize=(12, 10))
# ax2 = fig.add_subplot(111, projection="3d")
# for i, s in enumerate(src):
#     ax2.scatter(embedding_3d[Y_test==s, 0], 
#                 embedding_3d[Y_test==s, 1], 
#                 embedding_3d[Y_test==s, 2], 
#                 c=my_cols[i], 
#                 linewidths=0.1,
#                 edgecolors='k',
#                 label=s)
# plt.setp(ax2, xticks=[], yticks=[]) 
# plt.title("Julia source code data embedded into two dimensions by UMAP", 
#           fontsize=18) 
# plt.legend(loc="upper left", bbox_to_anchor=(1,1))
# plt.subplots_adjust(right=0.75)
# plt.show()


## Findings and Analysis
Encoded code snippets sort into several distinct groups based on syntactic and semantic content, but not necessarily in line with their source in the Julia source code directory tree. There is in fact a large degree of overlap across our directory-structure-defined categories. 
<img src="../img/julia_code_map100.png" />

Those in base/Base.jl exhibited a great degree of intra-group similarity and cohesion (this group had a silhouette score of 0.74 in the UMAP embedding space, and 0.39 in the auto encoded 64-dimensional representation space). The only other source code group with somewhat similar cohesion was base/docs/ (silhouette of 0.07 in UMAP space, and 0.28 in latent space), while the rest of the top level code groups were far more intermixed with each other.
<img src="../img/silhouettes.png" />

Using some quick k-means clustering on the UMAP embeddings allows us to zero in on some of these groups and inspect the code snippets for what common content and structures.
<img src="../img/k_clusts.png" />

Clusters 3, 15, 18, and 37 overlap most significantly with the bulk of code groups base_compiler, base_Base, base_boot, and base_docs, respectively. Looking at example snippets from these clusters shows some obvious consistencies in form and content. 

Cluster 3 is largely assignments, often of constants, and these would naturally feature prominently in the base_compiler code group. Referenceing our original UMAP projection graphic, it's also unsurprising to see many other code groups featured in cluster 3, as this is a very common type of expression across the Julia language. 

```Julia
primitive type UInt128 <: Unsigned 128 end
const UTF8PROC_COMPOSE = 1 << 3
const _tvarnames = Symbol[:_A, :_B, :_C, :_D, :_E, :_F, :_G, :_H, :_I, :_J, :_K, :_L, :_M, :_N, :_O, :_P, :_Q, :_R, :_S, :_T, :_U, :_V, :_W, :_X, :_Y, :_Z]
const LOG2_E = 1.4426950408889634
const DATATYPE_PARAMETERS_FIELDINDEX = fieldindex(DataType, :parameters)```


Cluster 15 is largely `include` statements, and again this makes intuitive sense that these would comprise much of the base_Base code snippets. This is very tightly grouped cluster, as these statements are syntacticly simple and very uniform. 

```Julia
include("strings/basic.jl")
include("process.jl")
include("gmp.jl")
include("arraymath.jl")
include("asyncmap.jl")```


Cluster 18, heavily identified with the base_boot code group, appears to consiste primarily of short type conversion functions. 

```Julia
toUInt16(x::UInt16) = begin
        x
    end

toUInt32(x::UInt8) = begin
        zext_int(UInt32, x)
    end

toInt16(x::Bool) = begin
        and_int(zext_int(Int16, x), Int16(1))
    end

toInt16(x::Int32) = begin
        checked_trunc_sint(Int16, x)
    end

toInt16(x::Int16) = begin
        x
    end```


Finally, Cluster 37, where we find most of our base_docs snippets, consists primarily of doc-string prefaced code snippets. The base_docs code group is almost all doc-strings itself, and thus the tight grouping of its code snippets in this cluster. 

```Julia
"""
    Signed <: Integer

Abstract supertype for all signed integers.
"""
Signed

"""
    copy(x)

Create a shallow copy of `x`: the outer structure is copied, but not all internal values.
For example, copying an array produces a new array with identically-same elements as the
original.
"""
copy

"""
    cis(z)

Return ``\\exp(iz)``.

# Examples
``jldoctest
julia> cis(π) ≈ -1
true
``
"""
function cis(z::Complex)
    v = exp(-imag(z))
    s, c = sincos(real(z))
    Complex(v * c, v * s)
end

"""
    InitError(mod::Symbol, error)

An error occurred when running a module's `__init__` function. The actual error thrown is
available in the `.error` field.
"""
InitError

"""
    StridedVecOrMat{T}

Union type of [`StridedVector`](@ref) and [`StridedMatrix`](@ref) with elements of type `T`.
"""
StridedVecOrMat
```

Finally, we look at different code snippets from the large agglomeration at the top of our UMAP visualization, represented in our cluster graphic as clusters 29, 13, 33, and 0 (from left to right). These clusters are all moderately short functions of similar complexity and layout, and these are found across all of our various code groups irrespective of purpose.

Cluster 29:
```Julia
function userefs(@nospecialize(x))
    relevant = x isa Expr && is_relevant_expr(x) || (x isa GotoIfNot || (x isa ReturnNode || (x isa PiNode || (x isa PhiNode || (x isa PhiCNode || x isa UpsilonNode)))))
    return UseRefIterator(x, relevant)
end

function adce_erase!(phi_uses, extra_worklist, compact, idx)
    if compact.result[idx] isa PhiNode
        maybe_erase_unused!(extra_worklist, compact, idx, (val->begin
                    phi_uses[val.id] -= 1
                end))
    else
        maybe_erase_unused!(extra_worklist, compact, idx)
    end
end```

Cluster 13:
```Julia
function gettypeinfos(io::IO, p::Pair)
    typeinfo = get(io, :typeinfo, Any)
    if p isa typeinfo <: Pair
        fieldtype(typeinfo, 1) => fieldtype(typeinfo, 2)
    else
        Any => Any
    end
end

function _maxlength(t::Tuple, t2::Tuple, t3::Tuple...)
    @_inline_meta
    max(length(t), _maxlength(t2, t3...))
end

function Base.deepcopy_internal(x::BigInt, stackdict::IdDict)
    if haskey(stackdict, x)
        return stackdict[x]
    end
    y = MPZ.set(x)
    stackdict[x] = y
    return y
end```

Cluster 33:
```Julia
function adduint64!(x::Bignum, operand::UInt64)
    operand == 0 && return
    other = Bignum()
    assignuint64!(other, operand)
    addbignum!(x, other)
end

function show(io::IO, r::LinRange)
    print(io, ""range("")
    show(io, first(r))
    print(io, "", stop="")
    show(io, last(r))
    print(io, "", length="")
    show(io, length(r))
    print(io, ')')
end```

Cluster 0:
```Julia
function length(s::AbstractString, i::Int, j::Int)
    @boundscheck begin
            0 < i ≤ ncodeunits(s) + 1 || throw(BoundsError(s, i))
            0 ≤ j < ncodeunits(s) + 1 || throw(BoundsError(s, j))
        end
    n = 0
    for k = i:j
        @inbounds n += isvalid(s, k)
    end
    return n
end

function fma(x::BigFloat, y::BigFloat, z::BigFloat)
    r = BigFloat()
    ccall(("mpfr_fma", :libmpfr), Int32, (Ref{BigFloat}, Ref{BigFloat}, Ref{BigFloat}, Ref{BigFloat}, MPFRRoundingMode), r, x, y, z, ROUNDING_MODE[])
    return r
end```


## Discussion and Future Avenues for Research
From this brief initial analysis, we can conclude a few things. First, as with other major programming languages previously tested, deep neural networks can in fact learn to recognize Julia code structures and content in potentially useful ways. Second, we confirm that UMAP clusters in our latent space successfully capture important characteristics of the code. And third - contrary to some of our prior expectations - while the latent representations of Julia source code capture important information and patterns in Julia expressions, these do not consistently point back to particular source code packages or groups in the Julia language ontology. Instead, these representations capture more low-level differencies and similarities, finding similar code structures that more often than not repeat across different regions of the code base. As with its human creators, there is much more difference within Julia source code groups than between them. 

Given these findings, it would be interesting to compare similar efforts with other languages to analyze how true this is across languages, or whether it is significantly more true in some languages than others. For instance, it would be interesting to explore if more parsimonious languages like Julia (or high level languages generally) exhibit the kind of wide disparity in syntactic patterns even within semantically similar code groups that we have seen here, while lower languages would see more syntactic similarity and semantic diversity. With respect to Julia code in particular and meta-modeling generally, we expect that further effort will enable us to leverage these embeddings for higher level tasks like recommending modifications, synthesizing models, identifying similar models, and validating the correctness of synthesized models.